In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from skimage.color import rgb2lab
from PIL import Image
import numpy as np

In [16]:
# Data Preparation
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([transforms.Resize((400, 400)),
                                    transforms.ToTensor()])
    img = transform(image)
    lab_image = rgb2lab(np.array(image))
    X = lab_image[:, :, 0]
    Y = lab_image[:, :, 1:] / 128.0
    X = X.reshape(1, 400, 400, 1)
    Y = Y.reshape(1, 400, 400, 2)
    return torch.Tensor(X), torch.Tensor(Y)

In [21]:
class ColorizationModel(nn.Module):
    def __init__(self):
        super(ColorizationModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Upsample(scale_factor=2, mode=;'nearest'),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode=;'nearest'),
            nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode=;'nearest'),
            nn.Conv2d(16, 2, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x * 128.0

In [22]:
model = Colorization()

# defining loss and optmizer
criterion = nn.MSELoss()
optimizer = optim.RMSprop(params=model.parameters(), lr=0.001) 

In [23]:
for epoch in range(1000):
    X, Y = preprocess_image('woman.jpg')

    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, Y)
    loss.backward()
    optimizer.step()


RuntimeError: Given groups=1, weight of size [8, 1, 3, 3], expected input[1, 400, 400, 1] to have 1 channels, but got 400 channels instead

In [24]:
with torch.no_grad():
    X, _ = preprocess_image('woman.jpg')
    output = model(X)
output *= 128
output = output.squeeze().permute(1, 2, 0).numpy()


RuntimeError: Given groups=1, weight of size [8, 1, 3, 3], expected input[1, 400, 400, 1] to have 1 channels, but got 400 channels instead